In [1]:
import pandas as pd
import Useful_functions_for_stations
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [2]:
bluebikes = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_idle_time_v4.csv')
bluebikes.head()

,start_time,end_time,start_station_id,end_station_id,bike_id,weekday,idle_time
0,2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,644,6,0
1,2017-01-01 00:13:16,2017-01-01 00:28:07,36,10,230,6,0
2,2017-01-01 00:16:17,2017-01-01 00:44:10,36,9,980,6,0
3,2017-01-01 00:21:22,2017-01-01 00:33:50,46,19,1834,6,0
4,2017-01-01 00:30:06,2017-01-01 00:40:28,10,8,230,6,0 days 00:01:59


In [3]:
bluebikes_stations = pd.read_csv('C:/Users/lauri/PycharmProjects/DSML_Data/bluebikes_stations.csv')
bluebikes_stations.head()

,station_id,lat,lon
0,67,42.358100,-71.093198
1,139,42.361780,-71.108100
2,36,42.349792,-71.077338
3,10,42.350406,-71.108279
4,9,42.351187,-71.115852


Density-based feature: Calculate density of stations within a certain radius. High density might imply high competition which could lead to higher idle times.

In [11]:
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic

In [8]:
geodata = gpd.GeoDataFrame(bluebikes_stations, geometry = gpd.points_from_xy(bluebikes_stations.lon, bluebikes_stations.lat))

geodata.set_crs(epsg=4326, inplace=True)

geodata = geodata.to_crs(epsg=3857)

buffer_radius = 1000 # = 1000 meter

geodata['buffer'] = geodata['geometry'].buffer(buffer_radius)
# Calculate number of stations within the radius around a station
geodata['station_density'] = geodata.apply(lambda row: geodata.within(row['buffer']).sum() - 1, axis=1)
# Convert station_density to per square km
#geodata['station_density'] = geodata['station_density'] / (3.1416* (buffer_radius/1000)**2)



In [9]:
geodata

,station_id,lat,lon,geometry,buffer,station_density
0,67,42.358100,-71.093198,POINT (-7914058.600 5214772.943),"POLYGON ((-7913058.600 5214772.943, -7913063.4...",3
1,139,42.361780,-71.108100,POINT (-7915717.430 5215327.402),"POLYGON ((-7914717.430 5215327.402, -7914722.2...",8
2,36,42.349792,-71.077338,POINT (-7912293.113 5213521.392),"POLYGON ((-7911293.113 5213521.392, -7911297.9...",9
3,10,42.350406,-71.108279,POINT (-7915737.409 5213613.945),"POLYGON ((-7914737.409 5213613.945, -7914742.2...",5
4,9,42.351187,-71.115852,POINT (-7916580.435 5213731.525),"POLYGON ((-7915580.435 5213731.525, -7915585.2...",4
...,...,...,...,...,...,...
308,229,42.345033,-71.096649,POINT (-7914442.783 5212804.682),"POLYGON ((-7913442.783 5212804.682, -7913447.5...",11
309,374,42.356683,-71.061666,POINT (-7910548.525 5214559.533),"POLYGON ((-7909548.525 5214559.533, -7909553.3...",14
310,376,42.360274,-71.128525,POINT (-7917991.131 5215100.408),"POLYGON ((-7916991.131 5215100.408, -7916995.9...",2
311,377,42.379273,-71.103419,POINT (-7915196.400 5217963.142),"POLYGON ((-7914196.400 5217963.142, -7914201.2...",7


Distance-based feature: There are important locations in Boston like the City_center or the universities. Bike stations closer to those locations might be used more frequently which could lead to lower idle times

In [12]:
# Important locations in Boston
locations = {
    'city_center': (42.3601, -71.0589),  # Boston City Center
    'logan_airport': (42.3656, -71.0096),  # Logan International Airport
    'boston_university': (42.3505, -71.1054),  # Boston University
    'fenway_park': (42.3467, -71.0972),  # Fenway Park
    'harvard_university': (42.3736, -71.1097)  # Harvard University
}
# Calculate distance to each location
for loc_name, loc_coord in locations.items():
    bluebikes_stations[loc_name + '_distance'] = bluebikes_stations.apply(lambda row: geodesic((row['lat'], row['lon']), loc_coord).km, axis=1)

bluebikes_stations

,station_id,lat,lon,geometry,city_center_distance,logan_airport_distance,boston_university_distance,fenway_park_distance,harvard_university_distance
0,67,42.358100,-71.093198,POINT (-71.09320 42.35810),2.834303,6.936987,1.312765,1.308539,2.193679
1,139,42.361780,-71.108100,POINT (-71.10810 42.36178),4.057397,8.125211,1.272618,1.900659,1.319524
2,36,42.349792,-71.077338,POINT (-71.07734 42.34979),1.902335,5.850388,2.313478,1.672191,3.755191
3,10,42.350406,-71.108279,POINT (-71.10828 42.35041),4.208350,8.302989,0.237444,1.001405,2.579055
4,9,42.351187,-71.115852,POINT (-71.11585 42.35119),4.795478,8.898705,0.864556,1.615649,2.540746
...,...,...,...,...,...,...,...,...,...
308,229,42.345033,-71.096649,POINT (-71.09665 42.34503),3.531929,7.526911,0.942697,0.190635,3.350412
309,374,42.356683,-71.061666,POINT (-71.06167 42.35668),0.442698,4.402141,3.668108,3.130701,4.380317
310,376,42.360274,-71.128525,POINT (-71.12852 42.36027),5.735832,9.814649,2.192808,2.989004,2.143764
311,377,42.379273,-71.103419,POINT (-71.10342 42.37927),4.240624,7.875336,3.200306,3.654337,0.815316
